# Multi-Agent System for Workshop Preparation

This notebook demonstrates how to build a multi-agent system using [smolagents](https://huggingface.co/docs/smolagents), where each agent performs a specialized task. The manager agent orchestrates their collaboration to assist users in preparing for datacraft workshops based on their learning goals.

## 1. Load dependencies and environment

We start by importing required packages and loading environment variables (such as the OpenAI API key).

In [7]:
import os
from dotenv import load_dotenv
from smolagents import CodeAgent, OpenAIServerModel

from workshop_smolagents_mcp.tools import query_events_db, web_search, summarize_url

load_dotenv(override=True)

True

## 2. Initialize the language model

We configure the OpenAI model that will be used across all agents.

In [8]:
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

## 3. Create specialized agents

We create two agents:
- `WebAgent` can search and summarize content from the web.
- `EventsAgent` can query a local SQLite database of datacraft workshops.

In [9]:
web_agent = CodeAgent(
    tools=[web_search, summarize_url],
    model=model,
    name="WebAgent",
    description="Can search the web and summarize content from online sources to help participants learn in advance about an event topic.",
)

events_agent = CodeAgent(
    tools=[query_events_db],
    model=model,
    name="EventsAgent",
    description="Can query the events database to find relevant datacraft workshops and trainings related to a user mention of an existing workshop.",
)

## 4. Create the manager agent

This agent coordinates the other two. It receives user goals and decides how to split the task between the database and web agents.

In [10]:
manager_agent = CodeAgent(
    tools=[],  # Required, even if empty
    model=model,
    managed_agents=[web_agent, events_agent],
    name="WorkshopPreparationAgent",
    description=(
        "An intelligent assistant that helps datacraft members learn in advance for upcoming workshops."
        "Given a user's workshop mention (e.g. 'I want to learn about next week's workshop on Watermarking'), it finds relevant workshops"
        "from the internal database and supplements them with external learning resources from the web to get started with the theory before attending the workshop."
    ),
)

## 5. Run an example request

We simulate a user asking to improve their skills in generative AI. The manager agent will find both workshop and web resources to match.

In [12]:
user_goal = "I think there is an event about learning about generative AI Watermarking with Hado, can you help me prepare for it?"
response = manager_agent.run(user_goal)
print(response)

╭────────────────────────────────────── New run - WorkshopPreparationAgent ───────────────────────────────────────╮
│                                                                                                                 │
│ I think there is an event about learning about generative AI Watermarking with Hado, can you help me prepare    │
│ for it?                                                                                                         │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            To prepare for the event about learning generative AI watermarking with Hado, I will perform the 
following steps:

1. **Event Details**: I will first check if there are specific details regarding this event such as the date, time,
venue, and a brief overview of what the event will cover.

2. **Content Preparation**: If the event details are found, I will gather content related to generative AI 
watermarking to help you prepare effectively for the event.

To start, I will query the `EventsAgent` to see if there is any event related to generative AI watermarking with 
Hado. Let's proceed with this.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 1.93 seconds| Input tokens: 2,176 | Output tokens: 130]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Summary of AI-powered invisible watermarking and generative AI                                                 
                                                                                                                   
  # AI-powered Invisible Watermarking:                                                                             
  #   - Purpose: Embedding digital watermarks covertly in AI-generated content to ensure provenance, deter         
  unauthorized use, and support copyright enforcement.                                                             
  #   - Techniques: Involves embedding digital signals invisible under normal viewing, detectable via specialized  
  algorithms; ensures authenticity, protects intellectual property, and enables traceability.                      
  #   - AI contributes robustness against removal and transformation in media.                                     
                                                                                                                   
  # Relevance to Generative AI:                                                                                    
  #   - Utilization of GANs among other AI techniques raises authenticity and IP concerns, addressed using         
  invisible watermarking.                                                                                          
  #   - Watermarking manages authenticity issues in AI-generated media by enhancing robustness against             
  manipulations such as resizing or compression.                                                                   
                                                                                                                   
  # Additional Considerations:                                                                                     
  #   - Includes digital rights management (DRM) and AI ethics.                                                    
  #   - Essential in maintaining legal and ethical digital media standards amidst growing AI content use.          
                                                                                                                   
  # This understanding aids in preparation for discussions and events related to generative AI watermarking        
  methodologies.                                                                                                   
  final_answer("Prepared for event with detailed understanding of AI-powered invisible watermarking in generative  
  AI contexts.")                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Prepared for event with detailed understanding of AI-powered invisible watermarking in generative AI 
contexts.

[Step 5: Duration 4.89 seconds| Input tokens: 15,563 | Output tokens: 1,046]

Prepared for event with detailed understanding of AI-powered invisible watermarking in generative AI contexts.
